# Risk Scoring & Underwriting
Risk Scoring and Underwriting are critical components in the financial and insurance industries, particularly in the evaluation and management of risk associated with lending, insurance policies, or other financial products.
## 1. Data Integration & Exploration

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import logging
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [2]:
# Define a class
class ARDPreprocessor:
    """
    Automated Risk Data (ARD) Preprocessing Module
    Handles missing values, encodes categorical variables, and scales numerical features.
    """
    # Constructor method to initialize the class
    def __init__(self, num_strategy='median', cat_strategy='most_frequent', scaler=StandardScaler(), encoder=OneHotEncoder(handle_unknown='ignore')):
        self.num_imputer = SimpleImputer(strategy=num_strategy)
        self.cat_imputer = SimpleImputer(strategy=cat_strategy)
        self.scaler = scaler
        self.encoder = encoder
        self.preprocessor = None

        # Configure logging to display messages at the INFO level and above
        logging.basicConfig(level=logging.INFO)

    # Define a fit method to fit the preprocessing pipeline to the input data
    def fit(self, df, numeric_features, categorical_features):
        # Begin a try block to handle potential errors during fitting
        try:
            # Create a Pipeline for numeric features
            numeric_pipeline = Pipeline([
                ('imputer', self.num_imputer),
                ('scaler', self.scaler)
            ])
            # Create a Pipeline for categorical features
            categorical_pipeline = Pipeline([
                ('imputer', self.cat_imputer),
                ('encoder', self.encoder)
            ])
            # Combine the numeric and categorical pipelines into a single preprocessing object
            self.preprocessor = ColumnTransformer([
                ('num', numeric_pipeline, numeric_features),
                ('cat', categorical_pipeline, categorical_features)
            ])
            # Fit the ColumnTransformer to the input DataFrame df
            self.preprocessor.fit(df)
            logging.info("Preprocessor fitted successfully.")
        except Exception as e:
            logging.error(f"Error during fitting: {e}")
            raise

    # Apply the fitted preprocessing pipeline to transform the input data
    def transform(self, df):
        try:
            transformed_data = self.preprocessor.transform(df)
            feature_names = self.preprocessor.get_feature_names_out()
            return pd.DataFrame(transformed_data, columns=feature_names)
        except Exception as e:
            logging.error(f"Error during transformation: {e}")
            raise

    # Combine fitting and transformation into a single method
    def fit_transform(self, df, numeric_features, categorical_features):
        self.fit(df, numeric_features, categorical_features)
        return self.transform(df)

The `ARDPreprocessor` class is a modular and reusable preprocessing module.

- It handles missing values, scales numeric features, and encodes categorical features.

- It uses `Pipeline` and `ColumnTransformer` for a clean and consistent workflow.

- It includes error handling and logging for robustness.

- The `fit_transform` method provides a convenient way to fit and transform data in one step.

Here's a breakdown of the steps:

1. Fitting:

- The `SimpleImputer` learns the mean (or other specified statistic) of the numeric features.

- The `StandardScaler` computes the mean and standard deviation of the numeric features.

2. Transformation:

- The `SimpleImputer` replaces missing values with the learned mean.

- The `StandardScaler` standardizes the features using the computed mean and standard deviation.

In [3]:
# Load and combine multiple datasets
file_paths = ['application_record.csv', 'credit_record.csv']
dfs = [pd.read_csv(file) for file in file_paths]

# Concatenate DataFrames into a single DataFrame
df = pd.concat(dfs, ignore_index=True)

This code snippet loads and combines multiple datasets from the specified file paths.

In [5]:
# Feature Engineering
df['income_log'] = np.log1p(df['AMT_INCOME_TOTAL'].replace(0, np.nan))  # Replace zero values before log transformation
df['age_years'] = (-df['DAYS_BIRTH']) // 365
df['employment_length_years'] = (-df['DAYS_EMPLOYED']).clip(lower=0) // 365  # Clip negative values
df['dependents'] = df['CNT_FAM_MEMBERS'] - df['CNT_CHILDREN']
df['has_car_property'] = ((df['FLAG_OWN_CAR'] == 'Y') | (df['FLAG_OWN_REALTY'] == 'Y')).astype(int)

In [6]:
# Creating Target Variable
def classify_risk(status):
    high_risk = {'2', '3', '4', '5'}
    low_risk = {'C', 'X', '0', '1'}
    return 1 if status in high_risk else 0

df['risk_flag'] = df['STATUS'].astype(str).apply(classify_risk)
logging.info("Target variable 'risk_flag' created based on STATUS")

In [9]:
# Check the shape of the dataset (rows, columns)
print("Dataset shape (rows, columns):\n", df.shape)

# Get dataset information (column names, non-null counts, data types)
print("Dataset Info:\n")
df.info()

# Display first 5 rows
print("\nFirst 5 rows:\n", df.head())

# Summary statistics for numerical columns
print("Summary statistics:\n", df.describe())

# Check for missing values in numerical columns
print("Missing values in numerical columns:\n", df.isnull().sum())

Dataset shape (rows, columns):
 (1487132, 26)
Dataset Info:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1487132 entries, 0 to 1487131
Data columns (total 26 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   ID                       1487132 non-null  int64  
 1   CODE_GENDER              438557 non-null   object 
 2   FLAG_OWN_CAR             438557 non-null   object 
 3   FLAG_OWN_REALTY          438557 non-null   object 
 4   CNT_CHILDREN             438557 non-null   float64
 5   AMT_INCOME_TOTAL         438557 non-null   float64
 6   NAME_INCOME_TYPE         438557 non-null   object 
 7   NAME_EDUCATION_TYPE      438557 non-null   object 
 8   NAME_FAMILY_STATUS       438557 non-null   object 
 9   NAME_HOUSING_TYPE        438557 non-null   object 
 10  DAYS_BIRTH               438557 non-null   float64
 11  DAYS_EMPLOYED            438557 non-null   float64
 12  FLAG_MOBIL               438557 non-n